In [ ]:
import os
import zipfile

def process_zip_and_create_symlink(zip_path):
    if not os.path.exists(zip_path):
        raise ValueError(f"Zip file does not exist: {zip_path}")

    if not zipfile.is_zipfile(zip_path):
        raise ValueError(f"Provided file is not a valid zip archive: {zip_path}")

    # Получение пути для распаковки
    extract_dir = os.path.join(os.getcwd(), os.path.basename(zip_path)[:-4])  # Убираем .zip из имени файла

    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)
        print(f"Created directory for extraction: {extract_dir}")

    # Распаковываем архив
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
        print(f"Extracted zip file into: {extract_dir}")

    # Автоматически ищем структуру COCO внутри распакованной директории
    def find_coco_structure(base_dir):
        for root, dirs, files in os.walk(base_dir):
            if "annotations" in dirs and "train2017" in dirs and "val2017" in dirs:
                return root
        return None

    coco_root = find_coco_structure(extract_dir)
    if not coco_root:
        raise ValueError("Extracted directory does not contain the expected COCO structure (annotations/, train2017/, val2017/).")

    print(f"Found COCO structure at: {coco_root}")

    # Путь к символической ссылке
    coco_symlink = "./YOLOX/datasets/COCO"

    # Создаем путь к папке, если он не существует
    coco_dir = os.path.dirname(coco_symlink)
    if not os.path.exists(coco_dir):
        os.makedirs(coco_dir)
        print(f"Created directory path: {coco_dir}")

    # Удаляем существующую символическую ссылку или директорию
    if os.path.islink(coco_symlink):
        os.unlink(coco_symlink)
        print(f"Removed symbolic link: {coco_symlink}")
    elif os.path.exists(coco_symlink):
        os.rmdir(coco_symlink)
        print(f"Removed directory: {coco_symlink}")

    # Создаем новую символическую ссылку
    os.symlink(coco_root, coco_symlink)
    print(f"Created symbolic link: {coco_symlink} -> {coco_root}")

# Пример использования
zip_file_path = "coco_ppe.zip"  # Замените на путь к вашему zip файлу
process_zip_and_create_symlink(zip_file_path)


In [ ]:
!python3 -m yolox.tools.train -expn "yolox_nano_rusal_ppe" -n yolox-nano -d 1 -b 64 --fp16 -o --cache ram max_epoch 300 print_interval 50 eval_interval 30 